In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping, log_evaluation
from sklearn.metrics import mean_squared_error

In [2]:
np.random.seed(23520945)
print(np.random.randint(1,100))

65


In [3]:
train = pd.read_csv('train_srhm.csv')

In [4]:
# Convert 'timestamp' to datetime and extract useful features
train['timestamp'] = pd.to_datetime(train['timestamp'])
train['year'] = train['timestamp'].dt.year
train['month'] = train['timestamp'].dt.month
train['day'] = train['timestamp'].dt.day
train.drop(columns=['timestamp'], inplace=True)  # Drop original timestamp

# Identify categorical columns
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()

# Apply Ordinal Encoding to categorical columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
train[categorical_cols] = encoder.fit_transform(train[categorical_cols])

# Replace boolean-like categorical columns with integers
boolean_cols = [col for col in train.columns if train[col].dtype == 'bool']
train[boolean_cols] = train[boolean_cols].astype(int)

# Handle missing values (Imputation with median)
imputer = SimpleImputer(strategy='median')
train = pd.DataFrame(imputer.fit_transform(train), columns=train.columns)

In [5]:
y = train['price_doc']
X = train.drop('price_doc', axis=1)   

In [6]:
# Assuming X and y are preprocessed
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate the model
lgbm = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    num_leaves=31,
    max_depth=7,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    bagging_freq=1,
    min_data_in_leaf=20
)

# Train the model
lgbm.fit(
    X_train, y_train, 
    eval_set=[(X_val, y_val)], 
    callbacks=[early_stopping(stopping_rounds=100), log_evaluation(period=100)]
)
# Make predictions and evaluate
y_pred = lgbm.predict(X_val)
rmse = mean_squared_error(y_val, y_pred) ** 0.5
print(f"RMSE: {rmse:.2f}")

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 9.86837e+12
[200]	valid_0's l2: 7.42674e+12
[300]	valid_0's l2: 6.67865e+12
[400]	valid_0's l2: 6.38954e+12
[500]	valid_0's l2: 6.22453e+12
[600]	valid_0's l2: 6.12074e+12
[700]	valid_0's l2: 6.05363e+12
[800]	valid_0's l2: 6.01262e+12
[900]	valid_0's l2: 5.98547e+12
[1000]	valid_0's l2: 5.98003e+12
Did not meet early stopping. Best iteration is:
[998]	valid_0's l2: 5.97865e+12
RMSE: 2445127.75


In [7]:
import optuna
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_squared_error

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),  # Fixed
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
    }

    model = LGBMRegressor(**param)
    
    # Corrected .fit() call
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], callbacks=[early_stopping(stopping_rounds=100)])
    
    preds = model.predict(X_val)
    rmse = mean_squared_error(y_val, preds) ** 0.5  # Compute RMSE manually
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2025-03-21 08:42:22,143] A new study created in memory with name: no-name-827df2a9-8fd9-454e-9d30-b8a42819d96d


[LightGBM] [Warning] feature_fraction is set=0.9311745302041092, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9311745302041092
[LightGBM] [Warning] min_data_in_leaf is set=32, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=32
Training until validation scores don't improve for 100 rounds


[I 2025-03-21 08:42:23,120] Trial 0 finished with value: 3899613.151109212 and parameters: {'n_estimators': 112, 'learning_rate': 0.002925791067777075, 'num_leaves': 35, 'max_depth': 9, 'min_data_in_leaf': 32, 'feature_fraction': 0.9311745302041092}. Best is trial 0 with value: 3899613.151109212.


Did not meet early stopping. Best iteration is:
[112]	valid_0's l2: 1.5207e+13
[LightGBM] [Warning] feature_fraction is set=0.9535645706079817, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9535645706079817
[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40
Training until validation scores don't improve for 100 rounds


[I 2025-03-21 08:42:27,689] Trial 1 finished with value: 3208860.0265138247 and parameters: {'n_estimators': 541, 'learning_rate': 0.001592649440210343, 'num_leaves': 44, 'max_depth': 7, 'min_data_in_leaf': 40, 'feature_fraction': 0.9535645706079817}. Best is trial 1 with value: 3208860.0265138247.


...

Early stopping, best iteration is:
[374]	valid_0's l2: 6.09219e+12
[LightGBM] [Warning] feature_fraction is set=0.7344860171645268, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7344860171645268
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
Training until validation scores don't improve for 100 rounds


[I 2025-03-21 08:45:49,641] Trial 98 finished with value: 2445605.1024942654 and parameters: {'n_estimators': 607, 'learning_rate': 0.04119234976722323, 'num_leaves': 26, 'max_depth': 10, 'min_data_in_leaf': 25, 'feature_fraction': 0.7344860171645268}. Best is trial 86 with value: 2421353.151272604.


Early stopping, best iteration is:
[282]	valid_0's l2: 5.98098e+12
[LightGBM] [Warning] feature_fraction is set=0.7583426312856536, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7583426312856536
[LightGBM] [Warning] min_data_in_leaf is set=29, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=29
Training until validation scores don't improve for 100 rounds


[I 2025-03-21 08:45:50,958] Trial 99 finished with value: 2508512.1293527773 and parameters: {'n_estimators': 700, 'learning_rate': 0.02288516211038906, 'num_leaves': 33, 'max_depth': 3, 'min_data_in_leaf': 29, 'feature_fraction': 0.7583426312856536}. Best is trial 86 with value: 2421353.151272604.


Did not meet early stopping. Best iteration is:
[691]	valid_0's l2: 6.29263e+12
